# 데이터 불러오기

In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 경로 설정       
os.chdir('/content/drive/My Drive/ESAA 데이터 분석 프로젝트 (가짜 뉴스 분류)/Data')
os.getcwd()

'/content/drive/.shortcut-targets-by-id/12KHC7G8PoQHHvGxvqV6nXOgUgwqOIO9O/ESAA 데이터 분석 프로젝트 (가짜 뉴스 분류)/Data'

In [5]:
import pandas as pd
train = pd.read_csv("news_train.csv")
test = pd.read_csv("news_test.csv")
train.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


# EDA

In [ ]:
# 그래프 한글 폰트 설정

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

%config InlineBackend.figure_format = 'retina'
!apt -qq -y install fonts-nanum

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

from IPython.display import set_matplotlib_formats
#폰트 선명하게 보이게 하기 위해
set_matplotlib_formats('retina')

## Test
plt.figure(figsize=(2,2))
plt.text(0.3, 0.3, '한글 폰트 테스트', size=15)
plt.show()

In [ ]:
# 기본 라이브러리
import numpy as np
from datetime import * 
import math

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#결측치 확인
np.sum(train['date'].isna()), np.sum(train['title'].isna()), np.sum(train['content'].isna()), np.sum(train['ord'].isna())

In [ ]:
real=train[train['info']==0] # 진짜 
fake=train[train['info']==1] # 가짜

진짜 뉴스와 가짜 뉴스의 분포

In [ ]:
print("진짜 뉴스 개수 : {}".format(len(train.loc[train['info']==0])))
print("가짜 뉴스 개수 : {}".format(len(train.loc[train['info']==1])))
print("진짜 뉴스 비율 : {}%".format(round((len(train.loc[train['info']==0])) / len(train.index) * 100, 3)))
print("가짜 뉴스 비율 : {}%".format(round((len(train.loc[train['info']==1])) / len(train.index) * 100, 3)))

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=train, x="info")

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(data=train, x='date', hue='info')

반복 등장 횟수 비교

In [ ]:
plt.figure(figsize=(12, 7))
sns.distplot(repe_fake['n_id'], label="가짜뉴스의 반복 등장 횟수")
sns.distplot(repe_real['n_id'], label="진짜뉴스의 반복 등장 횟수")
plt.xlim(0, 250)
plt.legend()

반복 등장하는 단어의 비교

In [ ]:
!pip install wordcloud
!pip install matplotlib
!pip install konlpy

In [8]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from konlpy.tag import Twitter
from collections import Counter

In [19]:
def make_wordcloud(data, word_count):
    twitter = Twitter()
    sentences_tag = []
    #형태소 분석하여 리스트에 넣기
    for sentence in data:
        morph = twitter.pos(sentence)
        sentences_tag.append(morph)
        
 
    noun_adj_list = []
    #명사와 형용사만 구분하여 이스트에 넣기
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun', 'Adjective']:
                noun_adj_list.append(word)
    
    #형태소별 count
    counts = Counter(noun_adj_list)
    tags = counts.most_common(word_count)
    print(tags)
 
    #wordCloud생성
    #한글깨지는 문제 해결하기위해 font_path 지정
    wc = WordCloud(font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', background_color='white', width=800, height=600)
    print(dict(tags))
    cloud = wc.generate_from_frequencies(dict(tags))
    plt.figure(figsize=(10, 8))
    plt.axis('off')
    plt.imshow(cloud)
    plt.show()

In [ ]:
make_wordcloud(fake['content'],100)

In [ ]:
make_wordcloud(real['content'],100)

In [ ]:
make_wordcloud(train['content'],100)

# Model

## Tokenizing 및 불용어 제거

### 형태소 분석기

Twitter, Komoran, Hannanum 등의 형태소 분석기(Pos Tagger)들이 존재. 
속도와 정확도 면에서 차이가 있으며 주로 Mecab 분석기를 이용함. 

Mecab: 굉장히 속도가 빠르면서도 좋은 분석 결과를 보여준다.

Komoran: 댓글과 같이 정제되지 않은 글에 대해서 먼저 사용해보면 좋다.(오탈자를 어느정도 고려해준다.)

Kkma: 분석 시간이 오래걸리기 때문에 잘 이용하지 않게 된다.

Okt: 품사 태깅 결과를 Noun, Verb등 알아보기 쉽게 반환해준다.

khaiii: 카카오에서 가장 최근에 공개한 분석기, 성능이 좋다고 알려져 있으며 다양한 실험이 필요하다.

형태소 분석기 비교 https://iostream.tistory.com/144 
https://mr-doosun.tistory.com/22

In [ ]:
#텍스트 예시
text = u"""ESAA 2조 프로젝트 파이팅 힘내요"""

In [ ]:
!pip3 install konlpy

In [ ]:
#Hannanum
from konlpy.tag import Hannanum
hannanum = Hannanum()

print(hannanum.analyze(text)) #다양한 형태로 변환(사전검색, 분류되지 않은 용어 등)
print(hannanum.morphs(text)) #텍스트에서 형태소 반환
print(hannanum.nouns(text))#텍스트에서 명사 반환
print(hannanum.pos(text))#텍스트에서 품사 정보 부착하여 반환

[[[('ESAA', 'f')]], [[('2', 'nnc'), ('조', 'nbu')], [('2', 'nnc'), ('조', 'nnc')]], [[('프로젝트', 'ncn')]], [[('파이팅', 'ncn')]], [[('힘내', 'pvg'), ('어요', 'ef')], [('힘내', 'pvg'), ('어', 'ef'), ('요', 'jxf')]]]
['ESAA', '2조', '프로젝트', '파이팅', '힘내', '어요']
['2조', '프로젝트', '파이팅']
[('ESAA', 'F'), ('2조', 'N'), ('프로젝트', 'N'), ('파이팅', 'N'), ('힘내', 'P'), ('어요', 'E')]


In [ ]:
#Komoran
from konlpy.tag import Komoran
komoran = Komoran()

print(komoran.morphs(text)) #텍스트에서 형태소 반환
print(komoran.nouns(text))#텍스트에서 명사 반환
print(komoran.pos(text))#텍스트에서 품사 정보 부착하여 반환

['ESAA', '2', '조', '프로젝트', '파이팅', '힘내', '요']
['프로젝트', '파이팅']
[('ESAA', 'SL'), ('2', 'SN'), ('조', 'NR'), ('프로젝트', 'NNP'), ('파이팅', 'NNP'), ('힘내', 'VV'), ('요', 'EC')]


In [ ]:
#Okt (과거 Twitter 분석기)
#from konlpy.tag import Twitter
from konlpy.tag import Okt

#twitter=Twitter()
okt = Okt()

print(okt.morphs(text)) #텍스트에서 형태소 반환
print(okt.nouns(text)) #텍스트에서 명사 반환
print(okt.pos(text)) #텍스트에서 품사 정보 부착하여 반환
print(okt.phrases(text)) #텍스트에서 어절을 뽑아냄

['ESAA', '2조', '프로젝트', '파이팅', '힘내요']
['프로젝트', '파이팅']
[('ESAA', 'Alpha'), ('2조', 'Number'), ('프로젝트', 'Noun'), ('파이팅', 'Noun'), ('힘내요', 'Verb')]
['ESAA', 'ESAA 2조', 'ESAA 2조 프로젝트', 'ESAA 2조 프로젝트 파이팅', '2조', '프로젝트', '파이팅']


#### MeCab 설치

In [ ]:
#Mecab
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

#출처: https://somjang.tistory.com/entry/Google-Colab에서-Mecab-koMecab-ko-dic-쉽게-사용하기 [솜씨좋은장씨]

In [ ]:
ls

In [ ]:
 cd Mecab-ko-for-Google-Colab

In [ ]:
! bash install_mecab-ko_on_colab190912.sh

In [ ]:
from konlpy.tag import Mecab 
mecab = Mecab()

print(mecab.nouns) #텍스트에서 명사 반환
print(mecab.morphs(text)) #텍스트에서 형태소 반환
print(mecab.pos(text)) #텍스트에서 품사 정보 부착하여 반환

#### khaiii(카카오) 설치

In [ ]:
#Khaiii
!git clone https://github.com/kakao/khaiii.git

In [ ]:
!pip install cmake

In [ ]:
!mkdir build

In [ ]:
!cd build && cmake /content/khaiii

In [ ]:
!cd /content/build/ && make all

In [ ]:
!cd /content/build/ && make resource

In [ ]:
!cd /content/build && make install

In [ ]:
!cd /content/build && make package_python

In [ ]:
!pip install /content/build/package_python

In [ ]:
from khaiii import KhaiiiApi
api = KhaiiiApi()
for word in api.analyze(text):
    print(word)

#출처: https://banana-media-lab.tistory.com/entry/colab에서-khaiii-형분석기-설치해서-사용하기 [Banana Media Lab]

### 스톱워드 제거

Mecab 사용

In [ ]:
from konlpy.tag import Mecab
import re
import tqdm
mecab  = Mecab()

def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Mecab()
    token_list = []
    
    for text in tqdm.tqdm(text_list):
        txt = re.sub('[^가-힣a-z]', ' ', text.lower()) 
        # 한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) # 형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] 
        #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
        
    return token_list, tokenizer

train['new_article'], mecab = text_preprocessing(train['content'])

Komoran 사용

In [ ]:
from konlpy.tag import Komoran
import re
import tqdm
komoran  = Komoran()

def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Komoran()
    token_list = []
    
    for text in tqdm.tqdm(text_list):
      try:
        txt = re.sub('[^가-힣a-z]', ' ', text.lower()) 
        # 한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) # 형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] 
        #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
      except:
        token_list.append(None)
    return token_list, tokenizer

train['new_article'], komoran = text_preprocessing(train['content'])

In [ ]:
train['new_article'].isnull().sum()

In [ ]:
train[train['new_article'].isnull() & train['info']==1]

In [ ]:
train[(train['new_article'].isnull()) & (train['info']==0)]

In [ ]:
# train으로 학습시킨 형태소 분석기 test에 적용
stopwords = ['을', '를', '이', '가', '은', '는', 'null']
token_list2 = []
for text in tqdm.tqdm(test['content']):
  try:
    txt2 = re.sub('[^가-힣a-z]', ' ', text.lower())
    token2 = komoran.morphs(txt2)
    token2 = [t for t in token2 if t not in stopwords or type(t) != float]
    token_list2.append(token2)
  except:
    token_list2.append(None)
test['new_article'] = token_list2

In [ ]:
test[(test['new_article'].isnull())]

In [ ]:
test[(test['new_article'].isnull()) & (test['content']==']]')]

In [ ]:
train_notnull = train[train['new_article'].notnull()]

새로운 stopword 사전

In [ ]:
stopword = pd.read_csv("stopwords.txt", sep='\t',names=['word', 'pos', 'val'], header=None)
stopword_new = stopword['word'] # 바꿔서 사용

## Vectorization

Vectorizer로 text2sequence 사용

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=1000):
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) 
    #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 
    #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) 
    #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_y = train_notnull['info']
train_X, vocab_size, vectorizer = text2sequence(train_notnull['new_article'], max_len = 100)
print(train_X.shape, train_y.shape)

In [ ]:
import numpy as np
np.save('train_X',train_X)

In [ ]:
test_notnull = test[test['new_article'].notnull()]

In [ ]:
# train으로 학습시킨 vectorizer test에도 적용
test_X_seq = vectorizer.texts_to_sequences(test_notnull['new_article'])
test_X = pad_sequences(test_X_seq, maxlen = 100)
#np.save('test_X', test_X)

## Embedding

#### word2vec

In [ ]:
!pip install --upgrade gensim

In [ ]:
#경로 다시 바꿔주기
os.chdir('/content/drive/My Drive/[데이콘] 한국어 문서 추출요약 AI 경진대회/Data')
os.getcwd()

In [ ]:
import gensim
import numpy as np

word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300))

In [ ]:
for index, word in enumerate(vectorizer.word_index):
    if word in word2vec:
        embedding_vector = word2vec[word] 
        embedding_matrix[index] = embedding_vector 
    else:
        print("word2vec에 없는 단어입니다.")
        continue

In [ ]:
embedding_matrix.shape

#### Fasttext

In [ ]:
# load the whole embedding into memory
from gensim.models.wrappers import FastText 
fastText = FastText.load_fasttext_format('fasttext.bin')
embedding_matrix = np.zeros((vocab_size, 100)) #300차원의 임베딩 매트릭스 생성
for index, word in enumerate(vectorizer.word_index): 
  #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in fastText: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = fastText[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_matrix[index] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.

In [ ]:
embedding_matrix.shape

## Modeling

### Auto ML -전처리X 

정확도 : 0.9458

In [ ]:
!pip install autokeras
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc4
!pip install autokeras
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak

In [ ]:
X = train.content.values
Y = train['info'].values

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=5
    )
clf.fit(X, Y, epochs=5)
model = clf.export_model()
model.summary()

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
pred_test = model.predict(test.content.values)
sample_submission.loc[:,'info'] = np.where(pred_test> 0.5, 1,0).reshape(-1)
sample_submission.loc[:,["id","info"]].to_csv("submission_AutoML1
.csv", index = False)
sample_submission

### Auto ML - vectorizer까지 진행

정확도 : 0.8497

In [ ]:
import tensorflow as tf
import autokeras as ak
input_node = ak.StructuredDataInput()
#output_node = ak.TextToIntSequence()(input_node)
#output_node = ak.Embedding()(output_node)
#output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()
clf = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node, 
    max_trials = 5, 
    overwrite=True)
clf.fit(train_X, train_y, epochs=5)
model = clf.export_model()


In [ ]:
pred_test = model.predict(test_X)
submission = pd.read_csv('sample_submission.csv')
submission.loc[:,'info'] = np.where(pred_test>0.5,1,0).reshape(-1)
submission.loc[:,['id','info']].to_csv("es_submission.csv", index=False)
submission

### RNN

정확도 : 0.9626

In [ ]:
X_data = train['content']
y_train = train['info']

tokenizer = Tokenizer(num_words = total_cnt - rare_cnt + 1)
tokenizer.fit_on_texts(X_data) # 5169개의 행을 가진 X의 각 행에 토큰화를 수행
sequences = tokenizer.texts_to_sequences(X_data) # 단어를 숫자값, 인덱스로 변환하여 저장
word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1
print('단어 집합의 크기: {}'.format((vocab_size)))

X_data = sequences
print('메일의 최대 길이 : %d' % max(len(l) for l in X_data))
print('메일의 평균 길이 : %f' % (sum(map(len, X_data))/len(X_data)))

max_len = 575 #메일의 최대 길이

# 전체 데이터셋의 길이는 max_len으로 맞춥니다.
X_train = pad_sequences(X_data, maxlen = max_len)
print("훈련 데이터의 크기(shape): ", data.shape)

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(vocab_size, 32)) # 임베딩 벡터의 차원은 32
model.add(SimpleRNN(32)) # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.2)

In [ ]:
testX_data = test['content']
sequences = tokenizer.texts_to_sequences(testX_data) # 단어를 숫자값, 인덱스로 변환하여 저장
X_data = sequences
print('메일의 최대 길이 : %d' % max(len(l) for l in X_data))
print('메일의 평균 길이 : %f' % (sum(map(len, X_data))/len(X_data)))
max_len = 300
# 전체 데이터셋의 길이는 max_len으로 맞춥니다.
data = pad_sequences(X_data, maxlen = max_len)
print("훈련 데이터의 크기(shape): ", data.shape)
sample_submission = pd.read_csv("sample_submission.csv")
pred_test = model.predict(data)
sample_submission.loc[:,'info'] = np.where(pred_test> 0.5, 1,0).reshape(-1)
sample_submission.loc[:,["id","info"]].to_csv("submission_RNN.csv", index = False)
sample_submission

### GRU

정확도 : 0.8527

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', 
                     mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(train_X, train_y, epochs=15, 
                    callbacks=[es, mc], batch_size=60, validation_split=0.2)
#에포크 5까지만 가고 멈춤

In [ ]:
loaded_model = load_model('best_model.h5')
sample_submission = pd.read_csv("sample_submission.csv")
pred_test = loaded_model.predict(test_X)
sample_submission.loc[:,'info'] = np.where(pred_test> 0.5, 1,0).reshape(-1)
sample_submission.loc[:,["id","info"]].to_csv("submission_GRU.csv", index = False)
sample_submission

### 1D CNN

임베딩 적용X + Mecab사용
정확도 : 0.97277

In [ ]:
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, 
Embedding, Dropout, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
 
model = Sequential()
model.add(Embedding(vocab_size, 32))
model.add(Dropout(0.2))
model.add(Conv1D(32, 5, strides=1, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', 
                     verbose = 1, save_best_only = True)

In [ ]:
history = model.fit(train_X, train_y, epochs = 10, batch_size=64, 
                    validation_split=0.2, callbacks=[es, mc])

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
pred_test = model.predict(test_X)
sample_submission.loc[:,'info'] = np.where(pred_test> 0.5, 1,0).reshape(-1)
sample_submission.loc[:,["id","info"]].to_csv("submission_1DCNN.csv", index = False)
sample_submission

임베딩O + Mecab 사용

정확도 : 0.9724

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], 
                    input_length = 1000)) # 여기만 바뀜
model.add(Dropout(0.2))
model.add(Conv1D(32, 5, strides=1, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', 
                     verbose = 1, save_best_only = True)
 
history = model.fit(train_X, train_y, epochs = 10, batch_size=64, 
                    validation_split=0.2, callbacks=[es, mc])

임베딩X + Mecab + 새로운 stopword 사전 사용

정확도 : 0.97072 

In [ ]:
stopword = pd.read_csv("stopwords.txt", sep='\t',names=['word', 'pos', 'val'], header=None)
stopword_new = stopword['word'] # 바꿔서 사용

In [ ]:
def text_preprocessing(text_list):
    
    tokenizer = Mecab()
    token_list = []
    
    for text in tqdm.tqdm(text_list):
      try:
        txt = re.sub('[^가-힣a-z]', ' ', text.lower()) 
        # 한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) # 형태소 분석
        token = [t for t in token if t not in stopword_new or type(t) != float] 
        #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
      except:
        token_list.append(None)
    return token_list, tokenizer

train['new_article'], komoran = text_preprocessing(train['content'])

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 32))
model.add(Dropout(0.2))
model.add(Conv1D(32, 5, strides=1, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)

history = model.fit(train_X, train_y, epochs = 10, batch_size=64, validation_split=0.2, callbacks=[es, mc])

## +임계값 조정

정확도 : 0.97277

In [ ]:
# 성능 지표 출력 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
 
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print("오차 행렬")
    print(confusion)
    print("정확도: {0:.5f}, 정밀도: {1:.5f}, 재현율: {2:.5f}, F1: {3:.5f}, AUC: {4:.5f}"
    .format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
from sklearn.preprocessing import Binarizer
 
# 평가지표를 조사하기 위한 새로운 함수 생성
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    #thresholds list 객체 내의 값을 iteration 하면서 평가 수행
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('\n임계값: ', custom_threshold)
        get_clf_eval(y_test, custom_predict, pred_proba_c1)


In [ ]:
# 임계값
thresholds = [0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56]
pred_proba = model.predict(train_X)
get_eval_by_threshold(train_y, pred_proba[:,0].reshape(-1, 1), thresholds)

임계값이 0.55일때 최고라 판정

In [ ]:
pred_test = model.predict(test_X)
test_notnull.loc[:,'info'] = np.where(pred_test> 0.55, 1,0).reshape(-1)

# 최종모델

기본 stopwords + Komoran + text2sequence + word2vec + 1D CNN

정확도 : 0.97297

In [ ]:
from konlpy.tag import Komoran
import re
import tqdm
komoran  = Komoran()
 
def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    tokenizer = Komoran()
    token_list = []
    
    for text in tqdm.tqdm(text_list):
      try:
        txt = re.sub('[^가-힣a-z]', ' ', text.lower()) 
        # 한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) # 형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] 
        #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
      except:
        token_list.append(None)
    return token_list, tokenizer
 
train['new_article'], komoran = text_preprocessing(train['content'])

In [ ]:
# train으로 학습시킨 형태소 분석기 test에 적용
stopwords = ['을', '를', '이', '가', '은', '는', 'null']
token_list2 = []
for text in tqdm.tqdm(test['content']):
  try:
    txt2 = re.sub('[^가-힣a-z]', ' ', text.lower())
    token2 = komoran.morphs(txt2)
    token2 = [t for t in token2 if t not in stopwords or type(t) != float]
    token_list2.append(token2)
  except:
    token_list2.append(None)
test['new_article'] = token_list2

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
train_notnull = train[train['new_article'].notnull()]
 
def text2sequence(train_text, max_len=1000):
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) 
    #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer
 
train_y = train_notnull['info']
train_X, vocab_size, vectorizer = text2sequence(train_notnull['new_article'], 
                                                max_len = 100)
print(train_X.shape, train_y.shape)

In [ ]:
test_notnull = test[test['new_article'].notnull()]
test_X_seq = vectorizer.texts_to_sequences(test_notnull['new_article'])
test_X = pad_sequences(test_X_seq, maxlen = 100)

In [ ]:
import gensim
import numpy as np
 
word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300))
for index, word in enumerate(vectorizer.word_index):
    if word in word2vec:
        embedding_vector = word2vec[word] 
        embedding_matrix[index] = embedding_vector 
    else:
        print("word2vec에 없는 단어입니다.")
        continue


In [ ]:
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, 
Embedding, Dropout, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
 
model = Sequential()
model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = 1000))
model.add(Dropout(0.2))
model.add(Conv1D(32, 5, strides=1, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
 
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', 
                     verbose = 1, save_best_only = True)
history = model.fit(train_X, train_y, epochs = 10, batch_size=64, 
                    validation_split=0.2, callbacks=[es, mc])
 
pred_test = model.predict(test_X)
test_notnull.loc[:,'info'] = np.where(pred_test> 0.5, 1,0).reshape(-1)
test_notnull['id'] = test_notnull['n_id']+"_"+test_notnull['ord'].map(str)
test_notnull.drop(['n_id', 'ord'], axis='columns', inplace=True)
 
test['id'] = test['n_id']+"_"+test['ord'].map(str)
test.drop(['n_id', 'ord'], axis='columns', inplace=True)
merge = pd.merge(test, test_notnull, how='outer', on='id')
merge = merge.loc[:,['id', 'info']]
merge


In [ ]:
# train에서 발견한 결과 적용하여 결측치 채우기
 
merge.fillna(0, inplace=True) # 숫자들은 모두 0
merge.at[29937, 'info']=1
merge.at[31326, 'info']=1 
# test 셋에서 확인 결과 이 두 인덱스의 content만 ‘]]’였음 (train에서 이를 1로 판별)